# Description

This notebook intends to gather all the functionalities you'll have to implement for assignment 2.1. You will have to generate an elastic solid, deform it, compute the associated Jacobian of the deformation map $\phi$, and implement pinning constraints. You will also visualize the eigenvectors and eigenvalues of the metric tensor, given a prescribed deformation.

# Load libraries

In [1]:
import numpy as np
import igl
import meshplot as mp

import sys as _sys
_sys.path.append("../src")
from elasticsolid import *
from eigendecomposition_metric import *

shadingOptions = {
    "flat":True,
    "wireframe":False,   
}

rot = np.array(
    [[1, 0, 0 ],
     [0, 0, 1],
     [0, -1, 0 ]]
)

# Load mesh

Several meshes are available for you to play with under `data/`: `ball.obj`, `dinosaur.obj`, and `beam.obj`. You can also uncomment the few commented lines below to manipulate a simple mesh made out of 2 tetrahedra.

In [2]:
v, _, _, t, _, _ = igl.read_obj("../data/dinosaur.obj")

# t = np.array([
#         [0, 1, 2, 3],
#         [1, 2, 3, 4]
#     ])
# v = np.array([
#     [0., 0., 0.],
#     [1., 0., 0.],
#     [0., 1., 0.],
#     [0., 0., 1.],
#     [2/3, 2/3, 2/3]
# ])

aabb = np.max(v, axis=0) - np.min(v, axis=0)
length_scale = np.mean(aabb)

p = mp.plot(v @ rot.T, t, shading=shadingOptions)

/opt/homebrew/anaconda3/envs/gc_course_env/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.987469…

# Manipulate elastic solids

## Instanciation

The rest shape matrices $D_m$ and their inverse matrices $B_m$ are computed during instanciation.

In [3]:
rho   = 131  # [kg.m-3]
solid = ElasticSolid(v, t, rho=rho)

## Deform the mesh

This part involves Jacobian computation which relies on deformed shape matrices $D_s$.

In [4]:
v_def = v.copy()
v_def[:, 2] *= 2.
solid.update_def_shape(v_def)

mp.plot(solid.v_def @ rot.T, solid.t, shading=shadingOptions)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.987469…

## Visualize some properties of the metric tensor

The metric tensor measures how stretched and sheared directions in the undeformed space are under the deformation $\phi$. It is defined from the Jacobian of the deformation $\mathbf{F}$ as follow (see the handout for a derivation):

$$\mathbf{M} = \mathbf{F}^T \mathbf{F}$$

We intend to plot the eigenvectors coloured by the corresponding eigenvalues in the next cell.

In [5]:
# We limit ourselves to stretching the mesh in the z direction
# Feel free to experiment with other kinds of deformations!

v_def = v.copy()
v_def[:, 2] *= 2.
solid.update_def_shape(v_def)

squared_eigvals, eigvecs = compute_eigendecomposition_metric(solid.F)
plot_eigendecomposition_metric(solid, squared_eigvals, eigvecs, rot, scale=0.05)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.987469…

## Pin vertices of the mesh

Pass a `pin_idx` to the constructor, compute the mask for deformations.

In [6]:
maxZ    = np.max(solid.v_rest[:, 2])
pin_idx = np.arange(solid.v_rest.shape[0])[solid.v_rest[:, 2] > maxZ - 0.1 * aabb[2]]

v_def = v.copy()
v_def[:, 2] -= 0.1 * aabb[2]
solid.update_def_shape(v_def)

solid_pinned = ElasticSolid(v, t, rho=rho, pin_idx=pin_idx)
solid_pinned.update_def_shape(v_def)

In [7]:
p = mp.plot(solid_pinned.v_def @ rot.T, t, shading=shadingOptions)
p.add_points(solid_pinned.v_def[pin_idx, :] @ rot.T, shading={"point_color":"black", "point_size": 0.1 * length_scale})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.987469…

1